In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'NanumGothic'
font_path = "./NanumGothic.ttf"

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt
from wordcloud import WordCloud

In [2]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud

In [ ]:
data = pd.read_csv('./t_data_2023_2022.csv') 
#-- 본인 환경에 맞게 설치 경로 변경할 것
df_data = pd.DataFrame(data)
df_data.head(10)

In [4]:
## 리뷰 데이터 탐색

In [ ]:
df_data.info()

# 코멘트가 없는 리뷰 데이터(NaN) 제거
df_reviews = df_data.dropna()

# 중복 리뷰 제거
df_reviews = df_reviews.drop_duplicates(['내용'])

df_reviews.info()
df_reviews.head(10)

# 연수명 리스트 확인
연수명_list = df_reviews.연수명.unique()

print('연수명_list =', len(연수명_list))
print(연수명_list[:10])

In [6]:
df_reviews = df_reviews['내용']

In [7]:
df_reviews = df_reviews.sample(1000) # 1000개 샘플링 

In [8]:
df_reviews.reset_index(drop=True, inplace=True)

In [ ]:
df_reviews

In [ ]:
df_reviews[0]+df_reviews[1]

In [11]:
reviews = ""

In [12]:
# 1000개의 리뷰 문장을 하나의 리뷰로 합치기 

for i in range(len(df_reviews)):
    reviews += df_reviews[i]

In [68]:
def apply_regular_expression(내용):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', 내용)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [ ]:
# 한글과 띄어쓰기를 제외한 문자 삭제 적용

reviews = apply_regular_expression(reviews)
reviews

In [ ]:
# 명사 형태소 추출 
okt = Okt()
nouns = okt.nouns(apply_regular_expression(reviews))
nouns

In [ ]:
# 명사 형태소 개수 확인 
from collections import Counter
counter = Counter(nouns)
counter

In [17]:
available_word = []

In [18]:
# 한글자 명사 제거 

for i in nouns:
    if len(i) > 1:
        available_word.append(i)

In [19]:
# 불용어 삭제
# 불용어 사전 불러오기 (https://www.ranks.nl/)

stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()

In [20]:
# 이외에 의미 없는 단어를 불용어에 추가 

tsherpa_stopwords = ['연수', '수업', '생각', '강의', '시간', '학생', '선생님', '내용', '활용',
                    '과학', '영어', '교사', '활동', '티셀파', '이야기', '아이', '대해', '통해', '대한', '부분']

In [21]:
for word in tsherpa_stopwords:
    stopwords.append(word)

In [22]:
clean_words = [] 

In [23]:
for i in available_word:
    if i not in stopwords:
        clean_words.append(i)

In [ ]:
clean_words

In [ ]:
pos_word_count = Counter(clean_words)
pos_word_count

In [56]:
type(pos_word_count)

collections.Counter

In [ ]:
pos_word_count = pd.DataFrame({'words':pos_word_count})

In [ ]:
pos_word_count

In [ ]:
max = 20
word_top_20 = {}
for word, counts in pos_word_count.most_common(20):
    word_top_20[word] = counts
    print(f'{word} : {counts}')

In [ ]:
plt.figure(figsize=(10, 5))
plt.title('티셀파 리뷰의 단어 상위 (%d개)' % max, fontsize=17)    
plt.ylabel('단어의 빈도수')    
plt.xticks(rotation=70)
for key, value in word_top_20.items():
    if key == '연수': continue
    plt.bar(key, value, color='lightgrey') 
plt.show()

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(font_path, background_color='ivory', width=800, height=600)
cloud = wc.generate_from_frequencies(pos_word_count)
plt.figure(figsize=(8,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()

In [29]:
import json
import pandas as pd

with open('data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f: 
    SentiWord_info = json.load(f)

sentiword_dic = pd.DataFrame(SentiWord_info)

In [30]:
# 유일값 저장
words_unique = list(set(clean_words))

In [31]:
df = pd.DataFrame(columns=("words", "sentiment"))

In [ ]:
sentiword_dic

In [33]:
sentiword_dic['polarity'].unique()

array(['1', '-1', '0', '-2', '2'], dtype=object)

In [34]:
len(words_unique)

2480

In [ ]:
words_unique

In [36]:
idx = 0                                             
 
for token in words_unique:
    sentiment = 0
    for i in range(0, len(sentiword_dic)):
        if sentiword_dic['word'][i] in token:
            sentiment = int(sentiword_dic['polarity'][i])
            df.loc[idx] = [token, sentiment]
            idx += 1

In [ ]:
df

In [38]:
len(df['words'].unique())

346

In [39]:
n_df = df[df.sentiment < 0]

In [40]:
p_df = df[df.sentiment > 0]

In [ ]:
n_df

In [42]:
n_df.rename(columns={'words':'n_words'}, inplace=True)

C:\Users\big-03-06\AppData\Local\Temp\ipykernel_5868\4007999875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_df.rename(columns={'words':'n_words'}, inplace=True)


In [43]:
n_df.rename(columns={'sentiment':'n_sentiment'}, inplace=True)

C:\Users\big-03-06\AppData\Local\Temp\ipykernel_5868\1205544233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  n_df.rename(columns={'sentiment':'n_sentiment'}, inplace=True)


In [ ]:
n_df

In [45]:
p_df.rename(columns={'words':'p_words'}, inplace=True)

C:\Users\big-03-06\AppData\Local\Temp\ipykernel_5868\546460829.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_df.rename(columns={'words':'p_words'}, inplace=True)


In [46]:
p_df.rename(columns={'sentiment':'p_sentiment'}, inplace=True)

C:\Users\big-03-06\AppData\Local\Temp\ipykernel_5868\265111265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_df.rename(columns={'sentiment':'p_sentiment'}, inplace=True)


In [ ]:
p_df

In [62]:
r_df=pd.concat([pos_word_count, p_df, n_df], axis=1)

In [ ]:
r_df

In [ ]:
plt.figure(figsize=(12,5))
X=[0,1]
plt.title('긍정과 부정 리뷰수' , fontsize=20)

plt.ylabel('리뷰수(개)' , fontsize=15)
plt.xlabel('T셀파' , fontsize=15)


r_df['p_sentiment'].value_counts().plot.bar(color = ['lightblue', 'tomato'])
ticklabel=['긍정 리뷰수','부정 리뷰수']

plt.xticks(X, ticklabel)

plt.show()

In [ ]:
plt.figure(figsize=(7,7))

plt.title('T셀파의 긍정과 부정 비율' , fontsize=20)

plt.ylabel('words' , fontsize=15)

r_df['p_sentiment'].value_counts().plot.pie(autopct = '%.2f%%' , 
                                         colors = ['lightblue', 'tomato'] ,
                                          labels = ['긍정', '부정'],
                                         textprops = {'fontsize' : 12 , 
                                                      'weight' : 'bold'})

plt.show()